In [ ]:
%pip install --upgrade modal
%pip install ipywidgets

In [ ]:
import modal

assert modal.__version__ > "0.49.0"
modal.__version__

In [ ]:
app = modal.App(name="example-basic")

### Handling standard Python functions

Standard Python functions can of course be defined in a notebook and used on their own or be called within Modal functions.
Below the `double` function is defined in pure-Python, and called once locally.

In [ ]:
def double(x: int) -> int:
    return x + x


double(5)

### Handling Modal Functions

If we wanted to run this trivial doubling function *in the cloud* we can write another function `double_with_modal` and decorate it with `@app.function` to register
the function with the Modal app.

To demonstrate that Modal functions you define in the notebook can be called by _other_ Modal functions, there's another function, `quadruple`, which uses `double` and `double_with_modal`.
For numbers greater than 1 million, this function spins up containers that run in Modal, which is a _very_ inefficient way to multiply a number by four, but you can do it if you please!

In [ ]:
@app.function()
def double_with_modal(x: int) -> int:
    return x + x


@app.function()
def quadruple(x: int) -> int:
    if x <= 1_000_000:
        return double(x) + double(x)
    else:
        return double_with_modal.remote(x) + double_with_modal.remote(x)


with app.run():
    print(quadruple.local(100))  # running locally
    print(quadruple.remote(100))  # run remotely
    print("Doing a very inefficient remote multiplication just for fun!")
    result = quadruple.remote(10_000_000)

In [ ]:
# Evaluate the result created in above cell
result

### GPU-powered notebook cells!

Thanks to Modal's remote execution capabilities, your notebook can be running on your laptop or a cheap CPU-only instance and take advantage of serverless GPU container execution. Here's the basics.

In [ ]:
# Define a Modal function with a GPU attached.
@app.function(gpu="any")
def hello_gpu():
    import subprocess

    subprocess.run("nvidia-smi", shell=True, check=True)
    return "hello from a remote GPU!"


# Start and run an ephemeral modal.App and execute the GPU-powered modal Function!
with app.run():
    result = hello_gpu.remote()
    assert result == "hello from a remote GPU!"

# After the app is finished you can continue executing other function's defined in your notebook and
# use the results of your GPU functions!
"This is the remote GPU's return value: " + result